In [1]:
# Import python library
import numpy as np
import mne
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# read raw SSVEP data
raw=mne.io.read_raw_curry('.\Data\SSVEP1.dat')

Leaving device<->head transform as None (no landmarks found)
Event file found. Extracting Annotations from C:\Users\OWNER\Desktop\Arthit Learn to Analysis EEG\SSVEP and EEGnet\Data\SSVEP1.ceo...


In [3]:
#pick only eeg
raw.pick_types(eeg=True)

<RawCurry | SSVEP1.dat, 122 x 458500 (458.5 s), ~116 kB, data not loaded>

In [4]:
#apply BP filter and nouch filter
raw.load_data().notch_filter(np.arange(50,500,50),verbose=False)
raw.filter(l_freq=0.1,h_freq=None,method='iir')

Reading 0 ... 458499  =      0.000 ...   458.499 secs...
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 0.10 Hz: -6.02 dB



<RawCurry | SSVEP1.dat, 122 x 458500 (458.5 s), ~426.9 MB, data loaded>

In [ ]:
# Annotation Function
#define duration each events
def find_net_annotation(raw_filted):
    raw=raw_filted
    for ann in range(17):
        raw_filted.annotations.duration[ann]=raw.annotations[ann+1]['onset']-raw.annotations[ann]['onset']
    # delete no need annotation 
    raw_filted.annotations.delete(np.arange(1,len(raw.annotations),2))
    raw_filted.annotations.onset
    #segmentation function
    def segment_eeg_data(start=None,end=None,duration=None,description=None):
        onset=list(np.arange(round(start),round(end),duration))
        return onset,[duration for j in range(len(onset))],[description for i in range(len(onset))]
    
    #declare variable
    onset=raw_filted.annotations.onset
    duration=raw_filted.annotations.duration
    labels=raw_filted.annotations.description
    end=np.round(onset+duration)
    onset_new=[]
    duration_new=[]
    descrip_new=[]
    for on,end,labs in zip(onset,end,labels):
        s,d,p=segment_eeg_data(start=on,end=end,duration=1,description=labs)
        onset_new.append(s)
        duration_new.append(d)
        descrip_new.append(p)
        
    #reshape data
    onset=[]
    duration=[]
    labels=[]
    for re in np.arange(len(onset_new)):
        onset=onset+onset_new[re]
        duration=duration+duration_new[re]
        labels=labels+descrip_new[re]
        
    #create annotation
    my_annot = mne.Annotations(onset=onset,  # in seconds
                               duration=duration,  # in seconds, too
                               description=labels)
    
    #Delete all old annotation
    raw_filted.annotations.delete(np.arange(0,len(raw_filted.annotations),1))
    
    #Set new annotation
    raw_filted.set_annotations(my_annot)
    return raw_filted

In [ ]:
raw=find_net_annotation(raw.copy())

## Epoch process 

In [ ]:
event,_id=mne.events_from_annotations(raw)

In [ ]:
pick_chs=[
    'P7',
 'P9',
 'P11',
 'PO11',
 'PO9',
 'PPO7',
 'P5',
 'P3',
 'PO3',
 'POO7',
 'POO9h',
 'POO11h',
 'OI1',
 'POO3',
 'PO1',
 'PPO1',
 'PPOz',
 'POz',
 'POOz',
 'Oz',
 'OI2',
 'POO4',
 'PO2',
 'PPO2',
 'P4',
 'PO4',
 'POO8',
 'POO10h',
 'POO12h',
 'PO12',
 'PO10',
 'PPO8',
 'P6',
 'P8',
 'P10',
 'P12',
          ]

In [ ]:
event_id={'10Hz':1,
          '20Hz':2,
          '30Hz':3
}

In [ ]:
epochs=mne.Epochs(raw,event,event_id,tmin=-0.1,tmax=1,picks=pick_chs)
del raw

In [ ]:
epochs.load_data().filter(l_freq=4,h_freq=40,method='iir')

In [ ]:
epochs.set_eeg_reference(ref_channels='average',verbose=False)

## Preprocess data 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,classification_report
## mne decoding library
from mne.decoding import Vectorizer,cross_val_multiscore,get_coef,Scaler

In [ ]:
X=epochs.get_data()
y=epochs.events[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
scaler=Scaler(epochs1.info)
scaler.fit(X_train,X_train.shape[0])
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

del epochs

##  EEGnet algorithm

In [ ]:
from EEGModels import EEGNet
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils as np_utils

In [ ]:
kernels,chans,samples=1,X.shape[1],X.shape[2]

In [ ]:
class_weigths={0:1, 1:1, 2:1, 3:1}

In [ ]:
yc_train = np_utils.to_categorical(y_train-1)
yc_test = np_utils.to_categorical(y_test-1)

In [ ]:
#reshape X
Xc_train=X_train.reshape(X_train.shape[0],chans,samples,kernels)
Xc_test=X_test.reshape(X_test.shape[0],chans,samples,kernels)

In [ ]:
model=EEGNet(nb_classes=3,Chans=chans,Samples=samples,dropoutRate=0.5,
            kernLength=250,F1=8,D=2,F2=16,dropoutType='Dropout')

In [ ]:
#model.compile(loss='CategoricalCrossentropy',optimizer='adam',metrics='accuracy')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
numParams=model.count_params()

In [ ]:
checkpointer=ModelCheckpoint(filepath='Users\OWNER\Desktop\Arthit Learn to Analysis EEG.h5',verbose=1,save_best_only=True)

In [ ]:
fittedmodel=model.fit(Xc_train,yc_train,batch_size=16,epochs=500,verbose=2,
                      validation_data=(Xc_test,yc_test),callbacks=[checkpointer],class_weight=class_weigths)

In [ ]:
model.load_weights('Users\OWNER\Desktop\Arthit Learn to Analysis EEG.h5')

In [ ]:
probs=model.predict(Xc_test)
preds=probs.argmax(axis=-1)
acc=np.mean(preds==yc_test.argmax(axis=-1))

In [ ]:
acc

In [ ]:
preds=preds+1

In [ ]:
preds

In [ ]:
print(confusion_matrix(y_test,preds))
print(classification_report(y_test,preds))

In [ ]:
preds

## plot History

In [ ]:
plt.plot(fittedmodel.history['accuracy'])
plt.plot(fittedmodel.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['Train','varidation'],loc='lower right')
plt.show()

In [ ]:
#loss
plt.plot(fittedmodel.history['loss'])
plt.plot(fittedmodel.history['val_loss'])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Train','varidation'],loc='lower right')
plt.show()

In [5]:
raw.get_data()

array([[ 1.10150405e-04,  1.10550472e-04,  1.44053361e-04, ...,
        -3.41800738e-05, -1.41459213e-05,  9.50308169e-07],
       [ 4.32177087e-05,  4.12637551e-05,  7.26559527e-05, ...,
        -2.46795593e-05, -1.49279428e-05, -4.78095089e-06],
       [ 1.12370571e-05,  1.70044167e-05,  5.00505306e-05, ...,
        -5.34684112e-06, -1.07805058e-05, -8.34536966e-06],
       ...,
       [ 4.64399551e-06, -1.30441068e-05,  3.62760068e-05, ...,
        -9.04508158e-06, -1.68329740e-05,  3.48026024e-06],
       [ 5.36105473e-06,  3.44322171e-05,  9.06772319e-05, ...,
        -1.41225937e-05, -9.91998152e-06,  5.18781490e-06],
       [ 2.67349622e-05,  9.64717411e-05,  1.13892765e-04, ...,
        -2.61154645e-05, -1.62073753e-05, -2.30435577e-06]])